<a href="https://colab.research.google.com/github/ekvirika/WalmartRecruiting/blob/main/notebooks/model_experiment_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required packages
!pip install wandb torch torchvision pandas numpy matplotlib seaborn scikit-learn

# Set up Kaggle API
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:

# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [4]:

# Download the dataset
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -q walmart-recruiting-store-sales-forecasting.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 492MB/s]


In [5]:
!unzip -q train.csv.zip
!unzip -q stores.csv.zip
!unzip -q test.csv.zip
!unzip -q features.csv.zip

unzip:  cannot find or open stores.csv.zip, stores.csv.zip.zip or stores.csv.zip.ZIP.


In [6]:
!pip install dagshub mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.6/680.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Experiment 1

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import xgboost as xgb
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from mlflow.models.signature import infer_signature
import warnings
warnings.filterwarnings('ignore')


In [8]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 18.9 MB/s eta 0:00:00


In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
features = pd.read_csv('features.csv')

In [10]:
# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(4, 1, figsize=(15, 12), sharex=True)

# # Original Weekly Sales
# axs[0].plot(train['Weekly_Sales'], label='Weekly_Sales', color='blue')
# axs[0].set_title('Original Weekly Sales')
# axs[0].legend()

# # Rolling Average with window=4
# axs[1].plot(train['Weekly_Sales_rolling_4'], label='Rolling Mean (4 weeks)', color='orange')
# axs[1].set_title('Rolling Average - Window 4')
# axs[1].legend()

# # Rolling Average with window=13
# axs[2].plot(train['Weekly_Sales_rolling_13'], label='Rolling Mean (13 weeks)', color='green')
# axs[2].set_title('Rolling Average - Window 13')
# axs[2].legend()

# # Rolling Average with window=52
# axs[3].plot(train['Weekly_Sales_rolling_52'], label='Rolling Mean (52 weeks)', color='red')
# axs[3].set_title('Rolling Average - Window 52')
# axs[3].legend()

# plt.tight_layout()
# plt.show()


In [11]:
import mlflow
from dagshub import dagshub_logger
import os

# Set tracking URI manually
mlflow.set_tracking_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

# Use your DagsHub credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "ekvirika"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "0adb1004ddd4221395353efea2d8ead625e26197"

# Optional: set registry if you're using model registry
mlflow.set_registry_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

In [12]:
# import pandas as pd
# import numpy as np
# import mlflow
# import mlflow.sklearn
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import RandomizedSearchCV
# import matplotlib.pyplot as plt
# import joblib



# # ----------------------
# # Evaluation Metric (WMAE)
# # ----------------------
# def weighted_mae(y_true, y_pred, is_holiday):
#     weights = np.where(is_holiday, 5, 1)
#     return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

# # ----------------------
# # Data Loader
# # ----------------------
# def load_data():
#     with mlflow.start_run(run_name="XGBoost_Data_Loading", nested=True):
#         train_df = pd.read_csv('train.csv')
#         test_df = pd.read_csv('test.csv')
#         stores_df = pd.read_csv('stores.csv')
#         features_df = pd.read_csv('features.csv')

#         df = train_df.merge(features_df, on=['Store', 'Date'], how='inner') \
#                      .merge(stores_df, on='Store', how='inner')

#         mlflow.log_params({
#             "train_shape": train_df.shape,
#             "test_shape": test_df.shape,
#             "missing_values_train": train_df.isnull().sum().sum(),
#             "missing_values_test": test_df.isnull().sum().sum()
#         })

#         return df

# # ----------------------
# # Preprocessing
# # ----------------------
# def preprocess(df):
#     with mlflow.start_run(run_name="XGBoost_Cleaning", nested=True):


#       df = df.copy()

#       # Fix column naming issues
#       df["IsHoliday"] = df.pop("IsHoliday_x") if "IsHoliday_x" in df else df["IsHoliday"]
#       df.drop(columns=["IsHoliday_y"], errors='ignore', inplace=True)

#       # Filter and sort
#       df = df[df["Weekly_Sales"] > 0]
#       df["Date"] = pd.to_datetime(df["Date"])
#       df = df.sort_values(by="Date")

#       # Remove outliers
#       q_low = df["Weekly_Sales"].quantile(0.01)
#       q_high = df["Weekly_Sales"].quantile(0.99)
#       df = df[(df["Weekly_Sales"] >= q_low) & (df["Weekly_Sales"] <= q_high)]

#       # Lag features
#       for lag in [1, 2, 4, 52]:
#           df[f"lag_{lag}"] = df.groupby("Store")["Weekly_Sales"].shift(lag)

#       # Rolling statistics
#       df["rolling_mean_4"] = df.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).mean()
#       df["rolling_std_4"] = df.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).std()

#       # Fill missing MarkDowns
#       markdown_cols = [col for col in df.columns if "MarkDown" in col]
#       df[markdown_cols] = df[markdown_cols].fillna(0)

#       # Date features
#       df["Year"] = df["Date"].dt.year
#       df["Month"] = df["Date"].dt.month
#       df["Week"] = df["Date"].dt.isocalendar().week
#       df["DayOfWeek"] = df["Date"].dt.dayofweek
#       df["IsMonthStart"] = df["Date"].dt.is_month_start.astype(int)
#       df["IsMonthEnd"] = df["Date"].dt.is_month_end.astype(int)
#       df["Quarter"] = df["Date"].dt.quarter
#       df.drop(columns=["Date"], inplace=True)

#       mlflow.log_params({"droppped_cols": "IsHoliday_x, IsHoliday_y, Date", "fill_markdown_NaNs": "0"})

#       # Drop rows with missing lag/rolling values
#       df = df.dropna()

#       return df

# # ----------------------
# # Dynamic Pipeline Builder
# # ----------------------
# def build_pipeline(X, model=None):
#     with mlflow.start_run(run_name="XGBoost_Feature_Engineering", nested=True):

#       numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
#       categorical_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

#       preprocessor = ColumnTransformer(transformers=[
#           ("num", SimpleImputer(strategy='mean'), numeric_cols),
#           ("cat", OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
#       ])

#       if model is None:
#           model = XGBRegressor(
#               n_estimators=100,
#               max_depth=6,
#               learning_rate=0.1,
#               random_state=42,
#               n_jobs=-1
#           )

#       pipeline = Pipeline([
#           ("preprocessor", preprocessor),
#           ("model", model)
#       ])

#       mlflow.log_params({"num_cols": numeric_cols, "cat_cols": categorical_cols,
#                         "simple_imputer": "mean", "cat_encoder": "one_hot", "pipeline": pipeline})

#       return pipeline

# # ----------------------
# # Runner
# # ----------------------
# def run():
#     # with mlflow.start_run(run_name="XGBoost_Training"):

#       df = load_data()
#       df = preprocess(df)

#       # Define features and target
#       X = df.drop(columns=["Weekly_Sales"])
#       y = df["Weekly_Sales"]

#       # Split
#       X_train, X_test, y_train, y_test = train_test_split(
#           X, y, test_size=0.2, random_state=42
#       )

#       # Store IsHoliday for WMAE
#       is_holiday_test = X_test["IsHoliday"].astype(bool).values if "IsHoliday" in X_test.columns else np.zeros_like(y_test)

#       # Train
#       pipeline = build_pipeline(X_train)

#       # Hyperparameter search space
#       param_dist = {
#           "model__n_estimators": [100, 200, 300],
#           "model__max_depth": [3, 6, 9],
#           "model__learning_rate": [0.05, 0.1, 0.2],
#           "model__subsample": [0.5, 0.7, 1.0],
#           "model__colsample_bytree": [0.5, 0.7, 1.0]
#       }

#       search = RandomizedSearchCV(
#           pipeline,
#           param_distributions=param_dist,
#           n_iter=20,
#           cv=3,
#           scoring="neg_mean_absolute_error",  # you could also define custom scorer using WMAE if needed
#           verbose=2,
#           n_jobs=-1,
#           random_state=42
#       )

#       search.fit(X_train, y_train)

#       best_model = search.best_estimator_
#       y_pred = best_model.predict(X_test)

#       # WMAE
#       wmae_score = weighted_mae(y_test.values, y_pred, is_holiday_test)
#       print(f"✅ WMAE: {wmae_score:.2f}")

#       mlflow.log_params(search.best_params_)
#       mlflow.log_metric("WMAE", wmae_score)
#       # mlflow.xgboost.log_model(best_model, artifact_path="xgb_model")
#       joblib.dump(best_model.named_steps["model"], "xgb_model.pkl")
#       mlflow.log_artifact("xgb_model.pkl")

#       joblib.dump(best_model, "xgb_pipeline.pkl")
#       mlflow.log_artifact("xgb_pipeline.pkl")

#       # Save feature importance plot as artifact
#       fig, ax = plt.subplots()
#       xgb_model = best_model.named_steps["model"]
#       importances = xgb_model.feature_importances_
#       print(importances)
#       ax.bar(range(len(importances)), importances)
#       plt.savefig("feature_importance.png")
#       mlflow.log_artifact("feature_importance.png")


#       return best_model

# # ----------------------
# # Entry Point
# # ----------------------
# if __name__ == "__main__":
#     try:
#         with mlflow.start_run(run_name="XGBoost_Training"):
#             model = run()
#         print("✅ Pipeline executed successfully!")
#     except Exception as e:
#         print(f"❌ Pipeline failed: {e}")


In [13]:
# import pandas as pd
# import numpy as np
# import mlflow
# import mlflow.sklearn
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.base import BaseEstimator, TransformerMixin
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_absolute_error
# import matplotlib.pyplot as plt
# import joblib
# import warnings
# from google.colab import files
# warnings.filterwarnings('ignore')

# # =====================================================
# # BASE TRANSFORMER CLASS
# # =====================================================
# class BaseWalmartTransformer(BaseEstimator, TransformerMixin):
#     """Base class for all Walmart transformers"""

#     def __init__(self):
#         self.is_fitted = False

#     def fit(self, X, y=None):
#         """Fit the transformer"""
#         self.is_fitted = True
#         return self

#     def transform(self, X):
#         """Transform the data"""
#         if not self.is_fitted:
#             raise ValueError("Transformer not fitted yet. Call fit() first.")
#         return X

#     def fit_transform(self, X, y=None):
#         """Fit and transform in one step"""
#         return self.fit(X, y).transform(X)

# # =====================================================
# # DATA LOADER CLASS
# # =====================================================
# class WalmartDataLoader:
#     """Handles loading and merging of Walmart datasets"""

#     def __init__(self, log_mlflow=True):
#         self.log_mlflow = log_mlflow
#         self.data_info = {}

#     def load_data(self, train_path='train.csv', test_path='test.csv',
#                   stores_path='stores.csv', features_path='features.csv'):
#         """Load and merge all datasets"""

#         if self.log_mlflow:
#             mlflow.start_run(run_name="Data_Loading", nested=True)

#         try:
#             # Load datasets
#             train_df = pd.read_csv(train_path)
#             test_df = pd.read_csv(test_path)
#             stores_df = pd.read_csv(stores_path)
#             features_df = pd.read_csv(features_path)

#             # Merge training data
#             merged_train = train_df.merge(features_df, on=['Store', 'Date'], how='inner') \
#                                   .merge(stores_df, on='Store', how='inner')

#             # Store data info
#             self.data_info = {
#                 "train_shape": train_df.shape,
#                 "test_shape": test_df.shape,
#                 "merged_train_shape": merged_train.shape,
#                 "missing_values_train": train_df.isnull().sum().sum(),
#                 "missing_values_test": test_df.isnull().sum().sum()
#             }

#             if self.log_mlflow:
#                 mlflow.log_params(self.data_info)
#                 mlflow.end_run()

#             print(f"Data loaded successfully!")
#             print(f"Training data shape: {merged_train.shape}")

#             return merged_train, test_df, stores_df, features_df

#         except Exception as e:
#             if self.log_mlflow:
#                 mlflow.end_run()
#             raise e

# # =====================================================
# # DATA CLEANING CLASS
# # =====================================================
# class WalmartDataCleaner(BaseWalmartTransformer):
#     """Handles data cleaning and basic preprocessing"""

#     def __init__(self, remove_outliers=True, outlier_quantiles=(0.01, 0.99), log_mlflow=True):
#         super().__init__()
#         self.remove_outliers = remove_outliers
#         self.outlier_quantiles = outlier_quantiles
#         self.log_mlflow = log_mlflow
#         self.q_low = None
#         self.q_high = None
#         self.cleaning_stats = {}

#     def fit(self, X, y=None):
#         """Fit the cleaner - compute outlier bounds"""
#         if self.log_mlflow:
#             mlflow.start_run(run_name="Data_Cleaning_Fit", nested=True)

#         try:
#             X = X.copy()

#             # Compute outlier bounds if Weekly_Sales exists
#             if 'Weekly_Sales' in X.columns and self.remove_outliers:
#                 self.q_low = X["Weekly_Sales"].quantile(self.outlier_quantiles[0])
#                 self.q_high = X["Weekly_Sales"].quantile(self.outlier_quantiles[1])

#                 self.cleaning_stats = {
#                     "outlier_lower_bound": self.q_low,
#                     "outlier_upper_bound": self.q_high,
#                     "original_rows": len(X)
#                 }

#             if self.log_mlflow:
#                 if self.cleaning_stats:
#                     mlflow.log_params(self.cleaning_stats)
#                 mlflow.end_run()

#             return super().fit(X, y)

#         except Exception as e:
#             if self.log_mlflow:
#                 mlflow.end_run()
#             raise e

#     def transform(self, X):
#         """Transform the data - clean and preprocess"""
#         super().transform(X)  # Check if fitted

#         if self.log_mlflow:
#             mlflow.start_run(run_name="Data_Cleaning_Transform", nested=True)

#         try:
#             X = X.copy()
#             original_rows = len(X)

#             # Fix IsHoliday column naming issues
#             if "IsHoliday_x" in X.columns:
#                 X["IsHoliday"] = X.pop("IsHoliday_x")
#             if "IsHoliday_y" in X.columns:
#                 X.drop(columns=["IsHoliday_y"], inplace=True)

#             # Filter positive sales (only if Weekly_Sales exists)
#             if 'Weekly_Sales' in X.columns:
#                 X = X[X["Weekly_Sales"] > 0]

#                 # Remove outliers if fitted bounds exist
#                 if self.remove_outliers and self.q_low is not None and self.q_high is not None:
#                     X = X[(X["Weekly_Sales"] >= self.q_low) & (X["Weekly_Sales"] <= self.q_high)]

#             # Convert Date to datetime and sort
#             if 'Date' in X.columns:
#                 X["Date"] = pd.to_datetime(X["Date"])
#                 X = X.sort_values(by="Date")

#             # Fill missing MarkDown values
#             markdown_cols = [col for col in X.columns if "MarkDown" in col]
#             if markdown_cols:
#                 X[markdown_cols] = X[markdown_cols].fillna(0)

#             rows_after_cleaning = len(X)

#             if self.log_mlflow:
#                 mlflow.log_params({
#                     "rows_before_cleaning": original_rows,
#                     "rows_after_cleaning": rows_after_cleaning,
#                     "rows_removed": original_rows - rows_after_cleaning,
#                     "markdown_cols_filled": len(markdown_cols)
#                 })
#                 mlflow.end_run()

#             print(f"Data cleaning completed: {original_rows} -> {rows_after_cleaning} rows")

#             return X

#         except Exception as e:
#             if self.log_mlflow:
#                 mlflow.end_run()
#             raise e

# # =====================================================
# # FEATURE ENGINEERING CLASS
# # =====================================================
# class WalmartFeatureEngineer(BaseWalmartTransformer):
#     """Handles feature engineering - lag features, rolling stats, date features"""

#     def __init__(self, lag_periods=[1, 2, 4, 52], rolling_windows=[4], log_mlflow=True):
#         super().__init__()
#         self.lag_periods = lag_periods
#         self.rolling_windows = rolling_windows
#         self.log_mlflow = log_mlflow
#         self.feature_stats = {}

#     def fit(self, X, y=None):
#         """Fit feature engineer - just store parameters"""
#         if self.log_mlflow:
#             mlflow.start_run(run_name="Feature_Engineering_Fit", nested=True)

#         try:
#             self.feature_stats = {
#                 "lag_periods": self.lag_periods,
#                 "rolling_windows": self.rolling_windows,
#                 "has_weekly_sales": 'Weekly_Sales' in X.columns,
#                 "has_date": 'Date' in X.columns,
#                 "has_store": 'Store' in X.columns
#             }

#             if self.log_mlflow:
#                 mlflow.log_params(self.feature_stats)
#                 mlflow.end_run()

#             return super().fit(X, y)

#         except Exception as e:
#             if self.log_mlflow:
#                 mlflow.end_run()
#             raise e

#     def transform(self, X):
#         """Transform data - create features"""
#         super().transform(X)

#         if self.log_mlflow:
#             mlflow.start_run(run_name="Feature_Engineering_Transform", nested=True)

#         try:
#             X = X.copy()
#             original_features = len(X.columns)

#             # Create lag features
#             if 'Weekly_Sales' in X.columns and 'Store' in X.columns:
#                 for lag in self.lag_periods:
#                     X[f"lag_{lag}"] = X.groupby("Store")["Weekly_Sales"].shift(lag)

#                 # Create rolling statistics
#                 for window in self.rolling_windows:
#                     X[f"rolling_mean_{window}"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=window).mean()
#                     X[f"rolling_std_{window}"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=window).std()

#             # Create date features
#             if 'Date' in X.columns:
#                 X["Year"] = X["Date"].dt.year
#                 X["Month"] = X["Date"].dt.month
#                 X["Week"] = X["Date"].dt.isocalendar().week
#                 X["DayOfWeek"] = X["Date"].dt.dayofweek
#                 X["IsMonthStart"] = X["Date"].dt.is_month_start.astype(int)
#                 X["IsMonthEnd"] = X["Date"].dt.is_month_end.astype(int)
#                 X["Quarter"] = X["Date"].dt.quarter

#                 # Drop Date column after feature extraction
#                 X.drop(columns=["Date"], inplace=True)

#             # Drop rows with NaN values (from lag/rolling features)
#             rows_before_dropna = len(X)
#             X = X.dropna()
#             rows_after_dropna = len(X)

#             new_features = len(X.columns)

#             if self.log_mlflow:
#                 mlflow.log_params({
#                     "original_features": original_features,
#                     "new_features": new_features,
#                     "features_added": new_features - original_features,
#                     "rows_before_dropna": rows_before_dropna,
#                     "rows_after_dropna": rows_after_dropna,
#                     "rows_dropped_due_to_nan": rows_before_dropna - rows_after_dropna
#                 })
#                 mlflow.end_run()

#             print(f"Feature engineering completed: {original_features} -> {new_features} features")
#             print(f"Rows after removing NaNs: {rows_before_dropna} -> {rows_after_dropna}")

#             return X

#         except Exception as e:
#             if self.log_mlflow:
#                 mlflow.end_run()
#             raise e

# # =====================================================
# # MODEL PIPELINE CLASS
# # =====================================================
# class WalmartModelPipeline(BaseWalmartTransformer):
#     """Handles model preprocessing and training"""

#     def __init__(self, model=None, log_mlflow=True):
#         super().__init__()
#         self.model = model
#         self.log_mlflow = log_mlflow
#         self.preprocessor = None
#         self.pipeline = None
#         self.numeric_cols = None
#         self.categorical_cols = None

#     def fit(self, X, y=None):
#         """Fit the preprocessing pipeline"""
#         if self.log_mlflow:
#             mlflow.start_run(run_name="Model_Pipeline_Fit", nested=True)

#         try:
#             # Identify column types
#             self.numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
#             self.categorical_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

#             # Create preprocessor
#             transformers = []
#             if self.numeric_cols:
#                 transformers.append(("num", SimpleImputer(strategy='mean'), self.numeric_cols))
#             if self.categorical_cols:
#                 transformers.append(("cat", OneHotEncoder(drop='first', handle_unknown='ignore'), self.categorical_cols))

#             self.preprocessor = ColumnTransformer(transformers=transformers)

#             # Create default model if none provided
#             if self.model is None:
#                 self.model = XGBRegressor(
#                     n_estimators=100,
#                     max_depth=6,
#                     learning_rate=0.1,
#                     random_state=42,
#                     n_jobs=-1
#                 )

#             # Create pipeline
#             self.pipeline = Pipeline([
#                 ("preprocessor", self.preprocessor),
#                 ("model", self.model)
#             ])

#             if self.log_mlflow:
#                 mlflow.log_params({
#                     "numeric_cols_count": len(self.numeric_cols),
#                     "categorical_cols_count": len(self.categorical_cols),
#                     "numeric_cols": self.numeric_cols[:10],  # Log first 10 to avoid too long
#                     "categorical_cols": self.categorical_cols,
#                     "model_type": type(self.model).__name__
#                 })
#                 mlflow.end_run()

#             return super().fit(X, y)

#         except Exception as e:
#             if self.log_mlflow:
#                 mlflow.end_run()
#             raise e

#     def transform(self, X):
#         """Transform using the fitted pipeline preprocessor"""
#         super().transform(X)

#         if self.pipeline is None:
#             raise ValueError("Pipeline not fitted yet.")

#         # Only transform using preprocessor, not the full pipeline
#         return self.pipeline.named_steps['preprocessor'].transform(X)

#     def get_pipeline(self):
#         """Get the full pipeline for training"""
#         if not self.is_fitted:
#             raise ValueError("Pipeline not fitted yet.")
#         return self.pipeline

# # =====================================================
# # EVALUATION METRICS CLASS
# # =====================================================
# class WalmartEvaluator:
#     """Handles model evaluation with WMAE and other metrics"""

#     def __init__(self):
#         pass

#     def weighted_mae(self, y_true, y_pred, is_holiday):
#         """Calculate Weighted Mean Absolute Error"""
#         weights = np.where(is_holiday, 5, 1)
#         return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

#     def evaluate_model(self, model, X_test, y_test, is_holiday_test=None):
#         """Evaluate model with multiple metrics"""
#         y_pred = model.predict(X_test)

#         # Standard MAE
#         mae_score = mean_absolute_error(y_test, y_pred)

#         # WMAE if holiday info available
#         wmae_score = None
#         if is_holiday_test is not None:
#             wmae_score = self.weighted_mae(y_test, y_pred, is_holiday_test)

#         metrics = {
#             'mae': mae_score,
#             'wmae': wmae_score
#         }

#         return metrics, y_pred

# # =====================================================
# # MAIN PIPELINE CLASS
# # =====================================================
# class WalmartSalesPipeline:
#     """Main pipeline orchestrating all components"""

#     def __init__(self, log_mlflow=True, experiment_name="Walmart_Sales_OOP"):
#         self.log_mlflow = log_mlflow
#         self.experiment_name = experiment_name

#         # Initialize components
#         self.data_loader = WalmartDataLoader(log_mlflow=log_mlflow)
#         self.data_cleaner = WalmartDataCleaner(log_mlflow=log_mlflow)
#         self.feature_engineer = WalmartFeatureEngineer(log_mlflow=log_mlflow)
#         self.model_pipeline = WalmartModelPipeline(log_mlflow=log_mlflow)
#         self.evaluator = WalmartEvaluator()

#         # Pipeline state
#         self.is_fitted = False
#         self.best_model = None
#         self.best_params = None
#         self.training_data = None

#         if self.log_mlflow:
#             mlflow.set_experiment(self.experiment_name)

#     def fit(self, train_path='train.csv', test_path='test.csv',
#             stores_path='stores.csv', features_path='features.csv',
#             hyperparameter_search=True, cv_folds=3, n_iter=20):
#         """Fit the entire pipeline"""

#         if self.log_mlflow:
#             mlflow.start_run(run_name="Walmart_Pipeline_Training")

#         try:
#             print("="*60)
#             print("WALMART SALES PIPELINE - TRAINING")
#             print("="*60)

#             # 1. Load data
#             print("\n1. Loading data...")
#             train_data, test_data, stores_data, features_data = self.data_loader.load_data(
#                 train_path, test_path, stores_path, features_path
#             )

#             # 2. Clean data
#             print("\n2. Cleaning data...")
#             self.data_cleaner.fit(train_data)
#             train_cleaned = self.data_cleaner.transform(train_data)

#             # 3. Engineer features
#             print("\n3. Engineering features...")
#             self.feature_engineer.fit(train_cleaned)
#             train_processed = self.feature_engineer.transform(train_cleaned)

#             # Store processed data
#             self.training_data = train_processed.copy()

#             # 4. Prepare features and target
#             print("\n4. Preparing features and target...")
#             X = train_processed.drop(columns=["Weekly_Sales"])
#             y = train_processed["Weekly_Sales"]

#             # 5. Split data
#             print("\n5. Splitting data...")
#             X_train, X_test, y_train, y_test = train_test_split(
#                 X, y, test_size=0.2, random_state=42
#             )

#             # Extract IsHoliday for WMAE
#             is_holiday_test = X_test["IsHoliday"].astype(bool).values if "IsHoliday" in X_test.columns else np.zeros_like(y_test, dtype=bool)

#             # 6. Fit model pipeline
#             print("\n6. Fitting model pipeline...")
#             self.model_pipeline.fit(X_train, y_train)
#             pipeline = self.model_pipeline.get_pipeline()

#             # 7. Hyperparameter tuning
#             if hyperparameter_search:
#                 print("\n7. Hyperparameter tuning...")
#                 param_dist = {
#                     "model__n_estimators": [100, 200, 300],
#                     "model__max_depth": [3, 6, 9],
#                     "model__learning_rate": [0.05, 0.1, 0.2],
#                     "model__subsample": [0.5, 0.7, 1.0],
#                     "model__colsample_bytree": [0.5, 0.7, 1.0]
#                 }

#                 search = RandomizedSearchCV(
#                     pipeline,
#                     param_distributions=param_dist,
#                     n_iter=n_iter,
#                     cv=cv_folds,
#                     scoring="neg_mean_absolute_error",
#                     verbose=1,
#                     n_jobs=-1,
#                     random_state=42
#                 )

#                 search.fit(X_train, y_train)
#                 self.best_model = search.best_estimator_
#                 self.best_params = search.best_params_

#                 if self.log_mlflow:
#                     mlflow.log_params(self.best_params)

#             else:
#                 print("\n7. Training model without hyperparameter search...")
#                 pipeline.fit(X_train, y_train)
#                 self.best_model = pipeline

#             # 8. Evaluate model
#             print("\n8. Evaluating model...")
#             metrics, y_pred = self.evaluator.evaluate_model(
#                 self.best_model, X_test, y_test, is_holiday_test
#             )

#             print(f"✅ MAE: {metrics['mae']:.2f}")
#             if metrics['wmae'] is not None:
#                 print(f"✅ WMAE: {metrics['wmae']:.2f}")

#             if self.log_mlflow:
#                 mlflow.log_metrics(metrics)

#             # 9. Save artifacts
#             print("\n9. Saving model artifacts...")
#             self._save_artifacts()

#             self.is_fitted = True

#             print("\n" + "="*60)
#             print("✅ PIPELINE TRAINING COMPLETED!")
#             print("="*60)

#             if self.log_mlflow:
#                 mlflow.end_run()

#             return self.best_model, metrics

#         except Exception as e:
#             if self.log_mlflow:
#                 mlflow.end_run()
#             raise e

#     def predict(self, new_data):
#         """Make predictions on new data"""
#         if not self.is_fitted:
#             raise ValueError("Pipeline not fitted yet. Call fit() first.")

#         test = pd.read_csv('test.csv')

#         # Apply same preprocessing steps
#         cleaned_data = self.data_cleaner.transform(new_data)
#         processed_data = self.feature_engineer.transform(cleaned_data)

#         # Remove target column if it exists
#         if 'Weekly_Sales' in processed_data.columns:
#             processed_data = processed_data.drop(columns=['Weekly_Sales'])

#         # Make predictions
#         predictions = self.best_model.predict(processed_data)

#         # Predict weekly sales
#         y_pred = self.best_model.predict(test)

#         test["Date"] = test["Date"].dt.strftime("%Y-%m-%d")

#         submission = test[["Id"]].copy()
#         submission["Weekly_Sales"] = y_pred

#         submission.to_csv("submission.csv", index=False)

#         files.download('submission.csv')

#         return predictions

#     def _save_artifacts(self):
#         """Save model artifacts"""
#         try:
#             # Save pipeline
#             joblib.dump(self.best_model, "walmart_pipeline.pkl")
#             if self.log_mlflow:
#                 mlflow.log_artifact("walmart_pipeline.pkl")

#             # Save individual components
#             joblib.dump(self.data_cleaner, "data_cleaner.pkl")
#             joblib.dump(self.feature_engineer, "feature_engineer.pkl")
#             joblib.dump(self.model_pipeline, "model_pipeline.pkl")

#             if self.log_mlflow:
#                 mlflow.log_artifact("data_cleaner.pkl")
#                 mlflow.log_artifact("feature_engineer.pkl")
#                 mlflow.log_artifact("model_pipeline.pkl")

#             # Feature importance plot
#             if hasattr(self.best_model.named_steps["model"], 'feature_importances_'):
#                 fig, ax = plt.subplots(figsize=(10, 6))
#                 importances = self.best_model.named_steps["model"].feature_importances_
#                 indices = np.argsort(importances)[::-1][:20]  # Top 20 features

#                 ax.bar(range(len(indices)), importances[indices])
#                 ax.set_title("Top 20 Feature Importances")
#                 ax.set_xlabel("Feature Index")
#                 ax.set_ylabel("Importance")

#                 plt.tight_layout()
#                 plt.savefig("feature_importance.png")

#                 if self.log_mlflow:
#                     mlflow.log_artifact("feature_importance.png")

#                 plt.close()

#         except Exception as e:
#             print(f"Warning: Could not save some artifacts: {e}")

# # =====================================================
# # USAGE EXAMPLE
# # =====================================================
# if __name__ == "__main__":
#     # Initialize and run pipeline
#     pipeline = WalmartSalesPipeline(
#         log_mlflow=True,
#         experiment_name="Walmart_Sales_OOP_Pipeline"
#     )

#     try:
#         # Train the pipeline
#         best_model, metrics = pipeline.fit(
#             hyperparameter_search=True,
#             cv_folds=3,
#             n_iter=10  # Reduced for faster testing
#         )

#         print(f"\n🎉 Training completed successfully!")
#         print(f"Best parameters: {pipeline.best_params}")

#     except Exception as e:
#         print(f"❌ Pipeline failed: {e}")
#         import traceback
#         traceback.print_exc()

In [14]:
# import pandas as pd
# import joblib
# import mlflow

# model_uri = "runs:/6d5792d20dbf4993b5753eb467020430/mlflow-artifacts:/e0cbe91d54c64e1d839ca8a3c7668be3/6d5792d20dbf4993b5753eb467020430/artifacts/model_pipeline.pkl"  # e.g., runs:/1234abcd/xgb_model
# model = mlflow.sklearn.load_model(model_uri)


# # Load your trained model
# # model = joblib.load("model.pkl")  # Or use the model directly if already in memory

# # Make predictions on test set
# preds = model.predict(test)

# # Construct the Id column — assumes you have these columns in your test set
# X_test_with_id = test.copy()
# X_test_with_id["Id"] = X_test_with_id["Store"].astype(str) + "_" + \
#                        X_test_with_id["Dept"].astype(str) + "_" + \
#                        X_test_with_id["Date"].astype(str)

# # Build submission DataFrame
# submission = pd.DataFrame({
#     "Id": X_test_with_id["Id"],
#     "Weekly_Sales": preds
# })

# # Save to CSV
# submission.to_csv("submission.csv", index=False)
# print("Submission file saved to 'submission.csv'")


In [41]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
features = pd.read_csv('features.csv')

In [42]:
from sklearn.base import BaseEstimator, TransformerMixin

class BaseMerger(BaseEstimator, TransformerMixin):

    def __init__(self, features_df, stores_df):
        self.features_df = features_df.copy()
        self.stores_df = stores_df.copy()
        self.features_df["Date"] = pd.to_datetime(self.features_df["Date"])
        self.stores_df["Store"] = self.stores_df["Store"].astype(int)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"])
        X["Store"] = X["Store"].astype(int)
        merged = X.merge(self.features_df, on=["Store", "Date"], how="left")
        merged = merged.merge(self.stores_df, on="Store", how="left")
        return merged

class FeatureAdder(BaseEstimator, TransformerMixin):

    def __init__(self, is_train=True):
        self.is_train = is_train

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"], errors="coerce")

        if self.is_train and "Weekly_Sales" in X.columns:
            for lag in [1, 2, 4, 52]:
                X[f"lag_{lag}"] = X.groupby("Store")["Weekly_Sales"].shift(lag)
            X["rolling_mean_4"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(4).mean()
            X["rolling_std_4"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(4).std()

        markdown_cols = [col for col in X.columns if "MarkDown" in col]
        X[markdown_cols] = X[markdown_cols].fillna(0)

        X["Year"] = X["Date"].dt.year
        X["Month"] = X["Date"].dt.month
        X["Week"] = X["Date"].dt.isocalendar().week
        X["DayOfWeek"] = X["Date"].dt.dayofweek
        X["IsMonthStart"] = X["Date"].dt.is_month_start.astype(int)
        X["IsMonthEnd"] = X["Date"].dt.is_month_end.astype(int)
        X["Quarter"] = X["Date"].dt.quarter

        # ⚠️ Don't drop Date here — it's needed for aggregations!
        return X

class MissingValueFiller(BaseEstimator, TransformerMixin):

    def __init__(self, is_train=True):
        self.q_low = None
        self.q_high = None
        self.is_train = is_train

    def fit(self, X, y=None):
        if self.is_train and "Weekly_Sales" in X.columns:
            self.q_low = X["Weekly_Sales"].quantile(0.01)
            self.q_high = X["Weekly_Sales"].quantile(0.99)
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"])

        if self.is_train and "Weekly_Sales" in X.columns:
            X = X[X["Weekly_Sales"] > 0]
            if self.q_low is not None and self.q_high is not None:
                X = X[(X["Weekly_Sales"] >= self.q_low) & (X["Weekly_Sales"] <= self.q_high)]

        return X


class CategoricalEncoder(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.fill_values = {}

    def fit(self, X, y=None):
        numeric_cols = X.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if X[col].isnull().any():
                self.fill_values[col] = X[col].mean()
        return self

    def transform(self, X):
        X = X.copy()

        for col, fill_value in self.fill_values.items():
            if col in X.columns:
                X[col] = X[col].fillna(fill_value)

        # if "IsHoliday" in X.columns:
        #     X["IsHoliday"] = X["IsHoliday"].astype(int)

        # X = X.dropna()  # Now okay, since lags are filled

        if "IsHoliday" in X.columns:
            X["IsHoliday"] = X["IsHoliday"].fillna(0).astype(int)  # Fill missing with 0 (not holiday)

        if "Type" in X.columns:
            X["Type"] = X["Type"].map({"A": 3, "B": 2, "C": 1}).fillna(0)

        # Fill missing values for numeric columns as before
        for col, fill_value in self.fill_values.items():
            if col in X.columns:
                X[col] = X[col].fillna(fill_value)

        # Now drop rows only if they still have missing values in important columns (exclude 'IsHoliday')
        X = X.dropna(subset=[col for col in X.columns if col != "IsHoliday"])

        return X


In [43]:
# Assuming df is your train.csv with 'Store' and 'Weekly_Sales'
train_dict = {}
for store_id, group in train.groupby("Store"):
    # You can define custom proportions here if needed
    train_dict[store_id] = (group.copy(), 1.0)


In [44]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_absolute_percentage_error

In [45]:
# # def train_store_models_xgb(train_weeks):
# #     mape_array = []
# #     store_models = {}

# #     for store in train_dict.keys():
# #         df, prop = train_dict[store]

# #         # Pipeline
# #         df = BaseMerger(features, stores).fit_transform(df)
# #         df = FeatureAdder(is_train=True).fit_transform(df)
# #         df = MissingValueFiller(is_train=True).fit_transform(df)
# #         df = CategoricalEncoder().fit_transform(df)

# #         df = df.sort_values("Date")
# #         train_df = df.iloc[:train_weeks]
# #         val_df = df.iloc[train_weeks:]

# #         X_train = train_df.drop(columns=["Weekly_Sales", "Date"])
# #         y_train = train_df["Weekly_Sales"]

# #         X_val = val_df.drop(columns=["Weekly_Sales", "Date"])
# #         y_val = val_df["Weekly_Sales"]

# #         model = XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=9)
# #         model.fit(X_train, y_train)

# #         preds = model.predict(X_val)
# #         mape = np.mean(np.abs((y_val - preds) / y_val)) * 100
# #         mape_array.append(mape)

# #         store_models[store] = (model, prop)

# #     return mape_array, store_models


# def train_store_models_xgb(train_weeks):
#     mape_array = []
#     wmae_array = []
#     store_models = {}

#     for store in train_dict.keys():
#         df, prop = train_dict[store]

#         # Pipeline
#         df = BaseMerger(features, stores).fit_transform(df)
#         df = FeatureAdder(is_train=True).fit_transform(df)
#         df = MissingValueFiller(is_train=True).fit_transform(df)
#         df = CategoricalEncoder().fit_transform(df)

#         df = df.sort_values("Date")
#         train_df = df.iloc[:train_weeks]
#         val_df = df.iloc[train_weeks:]

#         X_train = train_df.drop(columns=["Weekly_Sales", "Date"])
#         y_train = train_df["Weekly_Sales"]

#         X_val = val_df.drop(columns=["Weekly_Sales", "Date"])
#         y_val = val_df["Weekly_Sales"]

#         model = XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=9)
#         model.fit(X_train, y_train)

#         preds = model.predict(X_val)

#         # MAPE
#         mape = np.mean(np.abs((y_val - preds) / y_val)) * 100
#         mape_array.append(mape)

#         # WMAE (Holiday weight = 5, otherwise 1)
#         weights = np.where(val_df["IsHoliday"] == True, 5, 1)
#         wmae = np.sum(weights * np.abs(y_val - preds)) / np.sum(weights)
#         wmae_array.append(wmae)

#         store_models[store] = (model, prop)

#     return mape_array, wmae_array, store_models


import mlflow
import mlflow.xgboost
from xgboost import XGBRegressor

def train_store_models_xgb(train_weeks):
    mape_array = []
    wmae_array = []
    store_models = {}

    for store in train_dict.keys():
        df, prop = train_dict[store]

        df = df.sort_values("Date")

        # Pipeline
        df = BaseMerger(features, stores).fit_transform(df)
        df = FeatureAdder(is_train=True).fit_transform(df)
        df = MissingValueFiller(is_train=True).fit_transform(df)
        df = CategoricalEncoder().fit_transform(df)

        train_df = df.iloc[:train_weeks]
        val_df = df.iloc[train_weeks:]

        X_train = train_df.drop(columns=["Weekly_Sales", "Date"])
        y_train = train_df["Weekly_Sales"]

        # is_holiday_val = val_df["IsHoliday"]  # <-- Extract before dropping

        X_val = val_df.drop(columns=["Weekly_Sales", "Date"])
        y_val = val_df["Weekly_Sales"]

        model = XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=9)

        # Start MLflow run
        with mlflow.start_run(run_name=f"store_{store}"):
            mlflow.log_params({
                "store": store,
                "n_estimators": 300,
                "learning_rate": 0.1,
                "max_depth": 9,
            })

            model.fit(X_train, y_train)

            preds = model.predict(X_val)

            # MAPE
            mape = np.mean(np.abs((y_val - preds) / y_val)) * 100
            mape_array.append(mape)

            # WMAE
            if "IsHoliday" in val_df.columns:
                weights = np.where(val_df["IsHoliday"] == 1, 5, 1)
                print(weights)
            else:
                weights = np.ones(len(val_df))  # fallback equal weights

            wmae = np.sum(weights * np.abs(y_val - preds)) / np.sum(weights)
            wmae_array.append(wmae)

            mlflow.log_metrics({
                "MAPE": mape,
                "WMAE": wmae
            })

            # Log model
            # mlflow.xgboost.log_model(model, artifact_path="model")

            store_models[store] = (model, prop)

    return mape_array, wmae_array, store_models


In [46]:
train_weeks = 120
mape_array, wmae_array, store_models = train_store_models_xgb(train_weeks)

🏃 View run store_1 at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0/runs/db6d716509f64d749191a288001bef7e
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0
🏃 View run store_2 at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0/runs/c8001f5502c94caf94cbb7e3f015ec5a
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0
🏃 View run store_3 at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0/runs/8af286fdf5204e98bb5bf353bf6363b4
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0
🏃 View run store_4 at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0/runs/d8900454884240879d9984cd95b2525c
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0
🏃 View run store_5 at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0/runs/92f34dda08

In [47]:
submission = []

for store, (model, prop) in store_models.items():
    df, _ = train_dict[store]

    df = df.sort_values("Date")

    # Preprocess test data
    df = BaseMerger(features, stores).fit_transform(df)
    df = FeatureAdder(is_train=True).fit_transform(df)
    df = MissingValueFiller(is_train=True).fit_transform(df)
    df = CategoricalEncoder().fit_transform(df)

    val_df = df.iloc[train_weeks:]

    X_val = val_df.drop(columns=["Weekly_Sales", "Date"])
    dates = val_df["Date"]

    preds = model.predict(X_val)

    for i, row in val_df.iterrows():
        dept = prop["Dept"] if isinstance(prop, dict) and "Dept" in prop else 1
        prediction_date = row["Date"]
        submission.append({
            "Id": f"{store}_{dept}_{prediction_date.strftime('%Y-%m-%d')}",
            "Weekly_Sales": preds[i - val_df.index[0]]
        })

submission_df = pd.DataFrame(submission)
submission_df.to_csv("submission_xgb.csv", index=False)


IndexError: index 9918 is out of bounds for axis 0 with size 9918

In [50]:
# Load the real test.csv file
test = pd.read_csv("test.csv")

# Process and predict per store as you did for validation
for store, (model, prop) in store_models.items():
    test_store = test[test["Store"] == store].copy()

    test_store = BaseMerger(features, stores).fit_transform(test_store)
    test_store = FeatureAdder(is_train=True).fit_transform(test_store)
    test_store = MissingValueFiller(is_train=True).fit_transform(test_store)
    test_store = CategoricalEncoder().fit_transform(test_store)

    X_test = test_store.drop(columns=["Date"])
    dates = test_store["Date"]
    depts = test_store["Dept"]

    preds = model.predict(X_test)

    for i in range(len(test_store)):
        submission.append({
            "Id": f"{store}_{depts.iloc[i]}_{dates.iloc[i].strftime('%Y-%m-%d')}",
            "Weekly_Sales": preds[i]
        })


ValueError: feature_names mismatch: ['Store', 'Dept', 'IsHoliday_x', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday_y', 'Type', 'Size', 'lag_1', 'lag_2', 'lag_4', 'lag_52', 'rolling_mean_4', 'rolling_std_4', 'Year', 'Month', 'Week', 'DayOfWeek', 'IsMonthStart', 'IsMonthEnd', 'Quarter'] ['Store', 'Dept', 'IsHoliday_x', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday_y', 'Type', 'Size', 'Year', 'Month', 'Week', 'DayOfWeek', 'IsMonthStart', 'IsMonthEnd', 'Quarter']
expected lag_4, lag_1, lag_2, lag_52, rolling_std_4, rolling_mean_4 in input data

In [ ]:
from google.colab import files
files.download('submission_xgb.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GroupKFold, TimeSeriesSplit
import warnings
warnings.filterwarnings('ignore')

# Enhanced preprocessing classes for XGBoost
class BaseMerger(BaseEstimator, TransformerMixin):
    def __init__(self, features_df, stores_df):
        self.features_df = features_df.copy()
        self.stores_df = stores_df.copy()
        self.features_df["Date"] = pd.to_datetime(self.features_df["Date"])
        self.stores_df["Store"] = self.stores_df["Store"].astype(int)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"])
        X["Store"] = X["Store"].astype(int)
        merged = X.merge(self.features_df, on=["Store", "Date"], how="left")
        merged = merged.merge(self.stores_df, on="Store", how="left")
        return merged

class FeatureAdder(BaseEstimator, TransformerMixin):
    def __init__(self, is_train=True):
        self.is_train = is_train

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"], errors="coerce")

        # Department-level lag features (crucial for department predictions)
        if self.is_train and "Weekly_Sales" in X.columns:
            # Lag features grouped by Store and Dept
            for lag in [1, 2, 4, 8, 12, 52]:
                X[f"lag_{lag}"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(lag)

            # Rolling statistics
            X["rolling_mean_4"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1).rolling(4).mean()
            X["rolling_std_4"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1).rolling(4).std()
            X["rolling_mean_12"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1).rolling(12).mean()
            X["rolling_max_4"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1).rolling(4).max()
            X["rolling_min_4"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1).rolling(4).min()

            # Store-level aggregations (helpful for new departments)
            X["store_mean_sales"] = X.groupby(["Store", "Date"])["Weekly_Sales"].transform("mean")
            X["store_total_sales"] = X.groupby(["Store", "Date"])["Weekly_Sales"].transform("sum")

            # Department-level aggregations across stores
            X["dept_mean_sales"] = X.groupby(["Dept", "Date"])["Weekly_Sales"].transform("mean")

        # Handle markdown columns
        markdown_cols = [col for col in X.columns if "MarkDown" in col]
        X[markdown_cols] = X[markdown_cols].fillna(0)

        # Additional markdown features
        if len(markdown_cols) > 0:
            X["total_markdown"] = X[markdown_cols].sum(axis=1)
            X["markdown_count"] = (X[markdown_cols] > 0).sum(axis=1)

        # Enhanced date features
        X["Year"] = X["Date"].dt.year
        X["Month"] = X["Date"].dt.month
        X["Week"] = X["Date"].dt.isocalendar().week
        X["DayOfWeek"] = X["Date"].dt.dayofweek
        X["IsMonthStart"] = X["Date"].dt.is_month_start.astype(int)
        X["IsMonthEnd"] = X["Date"].dt.is_month_end.astype(int)
        X["Quarter"] = X["Date"].dt.quarter
        X["WeekOfMonth"] = (X["Date"].dt.day - 1) // 7 + 1
        X["DaysFromYear2010"] = (X["Date"] - pd.Timestamp('2010-01-01')).dt.days

        # Seasonal features
        X["sin_week"] = np.sin(2 * np.pi * X["Week"] / 52)
        X["cos_week"] = np.cos(2 * np.pi * X["Week"] / 52)
        X["sin_month"] = np.sin(2 * np.pi * X["Month"] / 12)
        X["cos_month"] = np.cos(2 * np.pi * X["Month"] / 12)

        return X

class MissingValueFiller(BaseEstimator, TransformerMixin):
    def __init__(self, is_train=True):
        self.q_low = None
        self.q_high = None
        self.is_train = is_train

    def fit(self, X, y=None):
        if self.is_train and "Weekly_Sales" in X.columns:
            self.q_low = X["Weekly_Sales"].quantile(0.005)  # More conservative outlier removal
            self.q_high = X["Weekly_Sales"].quantile(0.995)
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"])

        if self.is_train and "Weekly_Sales" in X.columns:
            # Remove negative sales
            X = X[X["Weekly_Sales"] > 0]
            # Remove extreme outliers
            if self.q_low is not None and self.q_high is not None:
                X = X[(X["Weekly_Sales"] >= self.q_low) & (X["Weekly_Sales"] <= self.q_high)]

        return X

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.fill_values = {}
        self.store_means = {}
        self.dept_means = {}

    def fit(self, X, y=None):
        # Calculate mean fill values
        numeric_cols = X.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if X[col].isnull().any():
                self.fill_values[col] = X[col].mean()

        # Calculate store and department level means for better imputation
        if "Weekly_Sales" in X.columns:
            self.store_means = X.groupby("Store")["Weekly_Sales"].mean().to_dict()
            self.dept_means = X.groupby("Dept")["Weekly_Sales"].mean().to_dict()

        return self

    def transform(self, X):
        X = X.copy()

        # Handle IsHoliday
        if "IsHoliday" in X.columns:
            X["IsHoliday"] = X["IsHoliday"].fillna(0).astype(int)

        # Handle Type with better encoding
        if "Type" in X.columns:
            X["Type"] = X["Type"].map({"A": 3, "B": 2, "C": 1}).fillna(0)

        # Fill missing values for numeric columns
        for col, fill_value in self.fill_values.items():
            if col in X.columns:
                X[col] = X[col].fillna(fill_value)

        # Smarter imputation for lag features using store/dept means
        lag_cols = [col for col in X.columns if 'lag_' in col or 'rolling_' in col]
        for col in lag_cols:
            if col in X.columns and X[col].isnull().any():
                # Fill with store-specific means where possible
                for store in X['Store'].unique():
                    if store in self.store_means:
                        mask = (X['Store'] == store) & X[col].isnull()
                        X.loc[mask, col] = self.store_means[store]

        # Final cleanup - drop rows with remaining missing values
        important_cols = [col for col in X.columns if col not in ["IsHoliday", "Date"]]
        X = X.dropna(subset=important_cols)

        return X

# XGBoost Model Wrapper for Store-Department combinations
class WalmartXGBoostModel:
    def __init__(self, store_id, dept_id, params=None):
        self.store_id = store_id
        self.dept_id = dept_id
        self.model = None
        self.feature_cols = None

        # Default XGBoost parameters optimized for retail sales
        self.params = {
            'n_estimators': 500,
            'learning_rate': 0.1,
            'max_depth': 6,
            'min_child_weight': 3,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'gamma': 0.1,
            'reg_alpha': 0.1,
            'reg_lambda': 1.0,
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': 0
        }

        if params:
            self.params.update(params)

    def fit(self, X, y, eval_set=None, early_stopping_rounds=50):
        """
        Fit XGBoost model
        """
        if len(X) < 10:  # Need minimum data points
            return None

        # Store feature columns
        self.feature_cols = [col for col in X.columns if col not in ['Date', 'Weekly_Sales']]

        # Initialize XGBoost
        self.model = XGBRegressor(**self.params)

        try:
            # Fit model
            if eval_set is not None:
                self.model.fit(
                    X[self.feature_cols], y,
                    eval_set=eval_set,
                    early_stopping_rounds=early_stopping_rounds,
                    verbose=False
                )
            else:
                self.model.fit(X[self.feature_cols], y)

            return self
        except Exception as e:
            print(f"Error fitting Store {self.store_id}, Dept {self.dept_id}: {str(e)}")
            return None

    def predict(self, X):
        """
        Make predictions using XGBoost model
        """
        if self.model is None or self.feature_cols is None:
            return None

        try:
            predictions = self.model.predict(X[self.feature_cols])
            return np.maximum(predictions, 0)  # Ensure non-negative
        except Exception as e:
            print(f"Error predicting Store {self.store_id}, Dept {self.dept_id}: {str(e)}")
            return np.zeros(len(X))

    def get_feature_importance(self):
        """
        Get feature importance from trained model
        """
        if self.model is None:
            return None

        importance = self.model.feature_importances_
        feature_importance = pd.DataFrame({
            'feature': self.feature_cols,
            'importance': importance
        }).sort_values('importance', ascending=False)

        return feature_importance

# Function to create store-department training data
def create_store_dept_data(train_df):
    """
    Create store-department level training data
    """
    store_dept_data = {}
    total_rows = len(train_df)

    for store_id in train_df['Store'].unique():
        store_data = train_df[train_df['Store'] == store_id].copy()
        store_dept_data[store_id] = {}

        for dept_id in store_data['Dept'].unique():
            dept_data = store_data[store_data['Dept'] == dept_id].copy()
            dept_data = dept_data.sort_values('Date').reset_index(drop=True)

            proportion = len(dept_data) / total_rows
            store_dept_data[store_id][dept_id] = (dept_data, proportion)

        print(f"Store {store_id}: {len(store_data['Dept'].unique())} departments, {len(store_data)} total records")

    return store_dept_data

# Training function with cross-validation
def train_xgboost_models_with_cv(store_dept_data, features, stores, cv_folds=3):
    """
    Train XGBoost models with cross-validation for each store-department combination
    """
    models = {}
    cv_scores = []
    successful_models = 0
    total_combinations = 0

    for store_id in store_dept_data.keys():
        print(f"\n=== Training Store {store_id} ===")
        models[store_id] = {}

        for dept_id in store_dept_data[store_id].keys():
            total_combinations += 1
            df, proportion = store_dept_data[store_id][dept_id]

            print(f"Training Store {store_id}, Dept {dept_id} ({len(df)} records)")

            try:
                # Apply preprocessing pipeline
                df = BaseMerger(features, stores).fit_transform(df)
                df = FeatureAdder(is_train=True).fit_transform(df)
                df = MissingValueFiller(is_train=True).fit_transform(df)
                df = CategoricalEncoder().fit_transform(df)

                # Sort by date
                df = df.sort_values("Date").reset_index(drop=True)

                if len(df) < 20:  # Need minimum data for reliable training
                    print(f"Skipping Store {store_id}, Dept {dept_id} - insufficient data ({len(df)} records)")
                    continue

                # Prepare features and target
                feature_cols = [col for col in df.columns if col not in ['Date', 'Weekly_Sales']]
                X = df[feature_cols]
                y = df['Weekly_Sales']

                # Time series cross-validation
                tscv = TimeSeriesSplit(n_splits=min(cv_folds, len(df)//10))
                fold_scores = []

                best_model = None
                best_score = float('inf')

                for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
                    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

                    # Train model
                    model = WalmartXGBoostModel(store_id, dept_id)
                    fitted_model = model.fit(
                        X_train, y_train,
                        eval_set=[(X_val[model.feature_cols], y_val)],
                        early_stopping_rounds=25
                    )

                    if fitted_model is not None:
                        # Validate
                        val_pred = fitted_model.predict(X_val)
                        if val_pred is not None:
                            # WMAE scoring (competition metric)
                            is_holiday = X_val['IsHoliday'].values if 'IsHoliday' in X_val.columns else np.zeros(len(X_val))
                            weights = np.where(is_holiday == 1, 5, 1)
                            wmae = np.sum(weights * np.abs(y_val - val_pred)) / np.sum(weights)
                            fold_scores.append(wmae)

                            if wmae < best_score:
                                best_score = wmae
                                best_model = fitted_model

                if best_model is not None and fold_scores:
                    avg_score = np.mean(fold_scores)
                    cv_scores.append(avg_score)

                    # Retrain on full data
                    final_model = WalmartXGBoostModel(store_id, dept_id)
                    final_fitted = final_model.fit(X, y)

                    if final_fitted is not None:
                        models[store_id][dept_id] = {
                            'model': final_fitted,
                            'cv_score': avg_score,
                            'proportion': proportion,
                            'training_size': len(df)
                        }
                        successful_models += 1
                        print(f"✓ Store {store_id}, Dept {dept_id} - CV WMAE: {avg_score:.2f}")

            except Exception as e:
                print(f"✗ Error training Store {store_id}, Dept {dept_id}: {str(e)}")
                continue

    print(f"\n=== Training Summary ===")
    print(f"Successfully trained: {successful_models}/{total_combinations} models")
    if cv_scores:
        print(f"Average CV WMAE: {np.mean(cv_scores):.2f} (+/- {np.std(cv_scores):.2f})")

    return models, cv_scores

# Generate predictions for test set
def generate_xgboost_submission(models, test_df, features, stores):
    """
    Generate predictions for test set in Walmart competition format
    """
    print("Generating test predictions...")

    # Apply preprocessing to test data
    test_processed = test_df.copy()
    test_processed = BaseMerger(features, stores).fit_transform(test_processed)
    test_processed = FeatureAdder(is_train=False).fit_transform(test_processed)
    test_processed = CategoricalEncoder().fit_transform(test_processed)

    submission_data = []
    predictions_made = 0
    fallback_predictions = 0

    # Group by store and department
    for (store_id, dept_id), group in test_processed.groupby(['Store', 'Dept']):
        group = group.sort_values('Date').reset_index(drop=True)

        # Check if we have a trained model
        if (store_id in models and dept_id in models[store_id]):
            model_info = models[store_id][dept_id]
            model = model_info['model']

            # Get predictions
            predictions = model.predict(group)

            if predictions is not None:
                predictions_made += len(predictions)
            else:
                predictions = np.full(len(group), 1000)  # Fallback
                fallback_predictions += len(predictions)
        else:
            # Fallback prediction strategy
            # Use department average from similar stores or overall average
            predictions = np.full(len(group), 1000)  # Simple fallback
            fallback_predictions += len(predictions)

        # Create submission entries
        for i, (_, row) in enumerate(group.iterrows()):
            submission_id = f"{int(row['Store'])}_{int(row['Dept'])}_{row['Date'].strftime('%Y-%m-%d')}"
            submission_data.append({
                'Id': submission_id,
                'Weekly_Sales': max(0, predictions[i])  # Ensure non-negative
            })

    print(f"Model predictions: {predictions_made}")
    print(f"Fallback predictions: {fallback_predictions}")
    print(f"Total predictions: {len(submission_data)}")

    return pd.DataFrame(submission_data)

# Feature importance analysis
def analyze_feature_importance(models, top_n=20):
    """
    Analyze feature importance across all models
    """
    all_importances = []

    for store_id in models.keys():
        for dept_id in models[store_id].keys():
            model_info = models[store_id][dept_id]
            importance = model_info['model'].get_feature_importance()

            if importance is not None:
                importance['store'] = store_id
                importance['dept'] = dept_id
                all_importances.append(importance)

    if all_importances:
        combined_importance = pd.concat(all_importances, ignore_index=True)

        # Calculate average importance across all models
        avg_importance = combined_importance.groupby('feature')['importance'].agg(['mean', 'std', 'count']).reset_index()
        avg_importance = avg_importance.sort_values('mean', ascending=False)

        print(f"\nTop {top_n} Most Important Features:")
        print(avg_importance.head(top_n))

        # Plot feature importance
        plt.figure(figsize=(12, 8))
        top_features = avg_importance.head(top_n)
        plt.barh(range(len(top_features)), top_features['mean'], xerr=top_features['std'])
        plt.yticks(range(len(top_features)), top_features['feature'])
        plt.xlabel('Average Feature Importance')
        plt.title(f'Top {top_n} Feature Importances Across All Models')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()

        return avg_importance

    return None

# Complete pipeline
def run_walmart_xgboost_pipeline(train_df, features, stores, test_df, cv_folds=3):
    """
    Complete XGBoost pipeline for Walmart competition
    """
    print("Starting Walmart XGBoost Pipeline...")

    # Step 1: Create store-department data
    print("\nStep 1: Creating store-department training data...")
    store_dept_data = create_store_dept_data(train_df)

    # Step 2: Train models with cross-validation
    print("\nStep 2: Training XGBoost models with cross-validation...")
    models, cv_scores = train_xgboost_models_with_cv(
        store_dept_data, features, stores, cv_folds
    )

    # Step 3: Analyze feature importance
    print("\nStep 3: Analyzing feature importance...")
    feature_importance = analyze_feature_importance(models)

    # Step 4: Generate test predictions
    print("\nStep 4: Generating test predictions...")
    submission_df = generate_xgboost_submission(models, test_df, features, stores)

    return models, submission_df, feature_importance

models, submission_df, feature_importance = run_walmart_xgboost_pipeline(
    train_df=train,
    features=features,
    stores=stores,
    test_df=test,
    cv_folds=3
)

# Save submission
submission_df.to_csv('walmart_xgboost_submission.csv', index=False)
print(f"Submission saved! Shape: {submission_df.shape}")
print("Sample predictions:")
print(submission_df.head(10))

In [ ]:
from google.colab import files
files.download('walmart_xgboost_submission.csv')